# SITCOM-1120: Brake analysis

# Data
#### e-stop - rolex
 * 22-11-2023
 1.  Fisrt log TMA:  2023-11-22T19:56:33.622523   /// bquint@love01.cp.lsst.org  /
 2.  by 2023-11-23T00:51:28 / running the TMA Az/El breaks tests( E-stop ) /// ashugart@love01.cp.lsst.org 
 3.  by 2023-11-23T02:32:17.254532 /    bquint@love01.cp.lsst.org 
   * We ran another TMA Brake Test earlier today. Taking note now. 2023-11-23 00:38 UTC - 2023-11-23 00:42 UTC
   * TMA Brake tests in Azimuth - Going from 151º to 0º
   * We stopped around 102.5º, and we overshot until 89.67º
 4. 2023-11-23T03:08:50.50 /   bquint@love01.cp.lsst.org 
   * - 2023-11-23 01:02 UTC - 2023-11-23 01:05 UTC
   * - TMA Brake tests in Elevation - Going up from 20º to 80º.
   * - We stopped around 40º, and we overshooted until 45º.
 5.  2023-11-23 01:35 UTC - 2023-11-23 01:40 UTC / by bquint@love01.cp.lsst.org 2023-11-23T03:10:09
   * - TMA Brake tests in Elevation - Going down from 80º to 20º.
   * - We stopped around 57.8º, and we overshooted until 53.8º.
    
---

 * BLOCK-142 : https://jira.lsstcorp.org/browse/BLOCK-142




In [ ]:
import sys, time, os, asyncio, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
from datetime import datetime
from astropy.time import Time, TimeDelta

from lsst_efd_client import EfdClient

client = EfdClient("usdf_efd")

%load_ext lab_black
%load_ext autoreload
%autoreload 2

# Functions

In [ ]:
def addTime(t_start, t_end):
    start = Time(t_start, scale="utc")
    end = Time(t_end, scale="utc")
    return (start, end)


def accel(v, t):
    a = np.gradient(v, t)
    return a


def plotAz(az, df_events):
    plt.figure(figsize=(16, 12))
    plt.suptitle("lsst.sal.MTMount.azimuth", fontsize=16)
    # Position
    plt.subplot(2, 1, 1)
    plt.plot(az["actualPosition"], label="Position")
    plt.plot(az["demandPosition"], label="demandPosition")
    plt.axvline(df_events)
    plt.xlabel("Time")
    plt.ylabel("Posición")
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # Velocity
    plt.subplot(2, 1, 2)
    plt.plot(az["actualVelocity"], label="Velocity")
    plt.plot(az["demandVelocity"], label="demandVelocity")
    plt.axvline(df_filter["active"])
    plt.xlabel("Time")
    plt.ylabel("Velocity")
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # disign
    plt.tight_layout()
    plt.show()
    return


def plotEl(el, df_events):
    plt.figure(figsize=(16, 12))
    # Common title for all subplots
    plt.suptitle("lsst.sal.MTMount.elevation", fontsize=16)
    # Position
    plt.subplot(2, 1, 1)
    plt.plot(el["actualPosition"], label="Position")
    plt.plot(el["demandPosition"], label="demandPosition")
    plt.axvline(df_events)
    plt.xlabel("Time")
    plt.ylabel("Posición")
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # Velocity
    plt.subplot(2, 1, 2)
    plt.plot(el["actualVelocity"], label="Velocity")
    plt.plot(el["demandVelocity"], label="demandVelocity")
    plt.axvline(df_events.all())
    plt.xlabel("Time")
    plt.ylabel("Velocity")
    plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # disign
    plt.tight_layout()
    plt.show()
    return


def errorPlot(df_filter):
    # plt.figure(figsize=(16, 12))
    # plt.suptitle("lsst.sal.MTMount.logevent_error", fontsize=16)
    # plt.plot
    # plt.eventplot(
    # df_filter["active"],
    # label="active",
    # )
    # plt.plot(logerror["code"], label="code")
    # plt.plot(logerror["latched"], label="latched")
    # plt.plot(logerror["subsystem"], label="subsystem")
    # plt.plot(logerror["text"], label="text")
    plt.axvline(df_filter["active"])
    # plt.xlabel("Time")
    # plt.ylabel("Posición")
    # plt.grid(":", alpha=0.5)  # or alpha=0.3
    plt.legend()

    # disign
    plt.tight_layout()
    plt.show()
    return


def clean_DF(df_az):
    """
    The function cleanDF takes a DataFrame df and a column name colname as input. Here's a description of what it does:

    Drop Column: It drops the column specified by colname from the DataFrame df.
    Reset Index: It resets the index of the DataFrame df after dropping the column.
    Rename Column: It renames the first column of the DataFrame df to colname.
    Convert to Datetime: It converts the values in the colname column to datetime format.
    Return: It returns the modified DataFrame with the specified column cleaned and formatted.

    """
    # Drop the 'timestamp' column from df DataFrame
    df_az = df_az.drop(columns=["timestamp"])
    df_az = df_az.reset_index()
    df_az = df_az.rename(columns={df_az.columns[0]: "timestamp"})

    # Convert the 'timestamp' column to datetime format
    df_az["timestamp"] = pd.to_datetime(df_az["timestamp"])

    return df_az


def filter_data_by_time_range(df, start_time, end_time):
    """
    This function filters the DataFrame 'df' based on the time range provided by a second 'obsTime'.
    Args:
        df (DataFrame): The DataFrame containing the data.
        obsTime (tuple): A tuple containing the start and end times of the desired range.
    Returns:
        DataFrame: The filtered DataFrame containing data within the specified time range.
    """
    # Convert the 'timestamp' column to Astropy Time objects
    df["timestamp"] = Time(df["timestamp"])

    # Filter the DataFrame by time range
    filtered_data = df[(df["timestamp"] >= start_time) & (df["timestamp"] <= end_time)]

    return filtered_data


def modif_Time(start_time, end_time):
    start_datetime = start_time.datetime
    end_datetime = end_time.datetime

    start_datetime = pd.Timestamp(start_datetime, tz="UTC")
    end_datetime = pd.Timestamp(end_datetime, tz="UTC")

    return start_datetime, end_datetime

# Running 

## Fesching data. # Main function:
 1.  Observing time started at : Fisrt log TMA:  2023-11-22T19:56:33.622523   /// bquint@love01.cp.lsst.org  /

#### **Don't run this section more than once. Shutdown kernel and clean outputs before start again.**

In [ ]:
obsTime = addTime("2023-11-22T20:00:00", "2023-11-23T04:00:00")

az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)


logerror = await client.select_time_series(
    "lsst.sal.MTMount.logevent_error",
    [
        "active",
        "code",
        "latched",
        "subsystem",
        "text",
    ],
    obsTime[0],
    obsTime[1],
)

####################################################################################
# AXIS Covert into dataFrame
df_az = pd.DataFrame(az)
df_el = pd.DataFrame(el)

# Data:
df_az = cleanDF(df_az)
df_el = cleanDF(df_el)

# Don't run this section more than once. Shutdown kernel and clean outputs before start again.

# Data Events

# Convert True and False to 1 y 0 - from "logerror"
df_error = pd.DataFrame(logerror)
df_error["active"] = df_error["active"].astype(int)

# Filter of the DataFrame, only values 0 and  1
df_filter = df_error[df_error["active"].isin([0, 1])]

df_events = df_filter[df_filter["active"] == 1].index

df_events = pd.DataFrame(df_events, columns=[""])
df_events = df_events.rename(columns={df_events.columns[0]: "timestamp"})
df_events["timestamp"] = pd.to_datetime(df_events["timestamp"])
# df_events

####################################################################################

### df_error : 
* **active**: IDL_Type: boolean / Units: unitless / Description: Is the error condition present?
* **text**: IDL_Type: string / Units: unitless / Description: Text description of err
* **subsystem**: IDL_Type:long / Units: unitless
* **Description**: Subsystem that produced the error; a single System enumeration value.

* **MTMount_logevent_elevationMotionState**: elevationMotionState ?? another keyword /  Description: Motion state of the elevation axis. / EFDB_Topic: MTMount_logevent_elevationMotionState




# Case 1:
 1.  by 2023-11-23T00:51:28 / running the TMA Az/El breaks tests( E-stop ) /// ashugart@love01.cp.lsst.org 


In [ ]:
time_start = "2023-11-23T00:40:00"
time_ends = "2023-11-23T00:55:00"

# To Time object
obsTime = addTime(time_start, time_ends)
start_time, end_time = obsTime

# Filtering Time eevents
# Due to the date type incompatibility, they need to be modified using modifTime() function . Convert to dadatime and ensure a UTC object.
start_datetime, end_datetime = modifTime(start_time, end_time)

# filtered_events = df_events[
#    (df_events["timestamp"] >= start_datetime)
#    & (df_events["timestamp"] <= end_datetime)
# ]

# Filter of the all data within the specified time range
filtered_events = filter_data_by_time_range(df_events, start_datetime, end_datetime)
filtered_data_az = filter_data_by_time_range(df_az, start_datetime, end_datetime)

# Plotting
# plotAz(filtered_data_az, filtered_events)

In [ ]:
filtered_events

In [ ]:
# # filtered_data_el = filter_data_by_time_range(df_el, obsTime[0], obsTime[1])

# Case 3:
 3.  by 2023-11-23T02:32:17.254532 /    bquint@love01.cp.lsst.org 
   * We ran another TMA Brake Test earlier today. Taking note now. 2023-11-23 00:38 UTC - 2023-11-23 00:42 UTC
   * TMA Brake tests in Azimuth - Going from 151º to 0º
   * We stopped around 102.5º, and we overshot until 89.67º


In [ ]:
obsTime = addTime("2023-11-23T00:30:00", "2023-11-23T00:50:00")

In [ ]:
# plotAz(az, acceleration=None)
plotAz(az)

# Case 4:
 4. 2023-11-23T03:08:50.50 /   bquint@love01.cp.lsst.org 
   * - 2023-11-23 01:02 UTC - 2023-11-23 01:05 UTC
   * - TMA Brake tests in Elevation - Going up from 20º to 80º.
   * - We stopped around 40º, and we overshooted until 45º.


In [ ]:
obsTime = addTime("2023-11-23T00:55:00", "2023-11-23T01:10:00")

az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

accAz = accel(az["actualVelocity"], az["timestamp"])
accEl = accel(el["actualVelocity"], el["timestamp"])

In [ ]:
# plotAz(az, acceleration=None)
plotAz(az, accAz)

In [ ]:
plotEl(el, accEl)

# Case 5:
 5.  2023-11-23 01:35 UTC - 2023-11-23 01:40 UTC / by bquint@love01.cp.lsst.org 2023-11-23T03:10:09
   * - TMA Brake tests in Elevation - Going down from 80º to 20º.
   * - We stopped around 57.8º, and we overshooted until 53.8º.

In [ ]:
obsTime = addTime("2023-11-23T00:30:00", "2023-11-23T01:00:00")

az = await client.select_time_series(
    "lsst.sal.MTMount.azimuth",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

el = await client.select_time_series(
    "lsst.sal.MTMount.elevation",
    [
        "actualPosition",
        "actualVelocity",
        "demandPosition",
        "demandVelocity",
        "actualAcceleration",
        "timestamp",
    ],
    obsTime[0],
    obsTime[1],
)

accAz = accel(az["actualVelocity"], az["timestamp"])
accEl = accel(el["actualVelocity"], el["timestamp"])

In [ ]:
# plotAz(az, acceleration=None)
plotAz(az, accAz)

In [ ]:
plotEl(el, accEl)

## Next:
1. ~~Summary state error , search for logs .. each one~~
2. Calculate distance after e-stop
3. ~~Search in GIS. / Search for FAULT~~
4. ~~Search for cmd_fault an E-Stop words?~~
5. Determind by diff ways the distance, -
6. Do it automatics the searsh of e-stops